# Clasificación de anuncios
El objetivo de este trabajo es lograr clasificar un datasets de anuncios publicitarios en base a caracteristicas de texto descriptivo de los objetos publicitados e imagines, para el cual se hace uso de tecnicas para el procesamiento de lenguaje natural a fin de lograr la clasificación. 

In [280]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 
import os 
from random import shuffle 
from tqdm import tqdm 
from PIL import Image
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
import pickle
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")


In [281]:
df = pd.read_csv("../input/desafio-dataset/ferramenta_competition_train.csv")
test_data = pd.read_csv('../input/desafio-dataset/ferramenta_competition_test_without_labels.csv')

In [282]:
df.head()

,image,title,description,class,f0,f1,f2,f3,f4,f5,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255
0,181587231.jpg,record irwin irwin record 96 morsa da banco pe...,irwin record 96 morsa da banco per tubi 3 152 mm,_@morsetto_strettoio_,0.000000,0.000000,0.000000,0.254326,0.000000,0.247400,...,0.237102,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.063287,0.000000
1,200447756.jpg,italchimica base tintometro x smalto fer plenu...,sistema tintometrico base per smalto antiruggi...,_@vernice_,0.420585,0.000000,0.000000,0.454027,0.000000,0.796165,...,0.586435,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,160784717.jpg,maurer cerniera per mobili perno fisso art 500...,confezione 4 pzmisura 1 3 4misura 2 20x20 mm. ...,_@cerniera_bandella_,0.105055,0.512814,0.000000,0.309017,0.000000,0.519669,...,0.565308,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.617254,0.000000
3,115047083.jpg,sanifri 470018043 connettori ad angolo zincati...,sanifri 470018043 connettori ad angolo zincati...,_@staffa_squadretta_,0.052270,0.000000,0.000000,0.006339,0.000000,0.204171,...,0.000000,0.223212,0.0,0.0,0.0,0.0,0.0,0.0,0.019491,0.000000
4,171654232.jpg,weller 2020 saldatore elettrico per ferro 20 w...,weller 2020 saldatore elettrico per ferro 20 w...,_@accessori_saldatore_,0.481601,0.500977,0.133263,0.558505,0.042294,0.000000,...,0.642299,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.304132,0.006778


In [283]:
df.groupby('class').count()["title"].plot(kind='bar')


<IPython.core.display.Javascript object>

<AxesSubplot:xlabel='class'>

In [284]:
clases = np.unique([str(item) for item in df['class'].values])

len(clases)

16

In [285]:
train_images = '../input/desafio-dataset/ferramenta-competition-images/ferramenta-competition-images/ferramenta-competition-images'
image_size = 16

In [313]:
def train_images_data(df):
    train_images_data = []
    for image1 in df['image']: 
        path = os.path.join(train_images, image)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        train_images_data.append(img1) 
    return train_images_data

In [287]:
data['im_vec'] = data.apply(lambda x: list(x[3:258]), axis= 1)
data.head()

,title,description,class,f0,f1,f2,f3,f4,f5,f6,...,f249,f250,f251,f252,f253,f254,f255,im_vec,prep_title,prep_desc
0,record irwin irwin record 96 morsa da banco pe...,irwin record 96 morsa da banco per tubi 3 152 mm,_@morsetto_strettoio_,0.000000,0.000000,0.000000,0.254326,0.000000,0.247400,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.063287,0.000000,"[0.0, 0.0, 0.0, 0.2543257, 0.0, 0.24739994, 0....","[record, irwin, irwin, record, morsa, banco, t...","[irwin, record, morsa, banco, tubi]"
1,italchimica base tintometro x smalto fer plenu...,sistema tintometrico base per smalto antiruggi...,_@vernice_,0.420585,0.000000,0.000000,0.454027,0.000000,0.796165,0.015496,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,"[0.4205849, 0.0, 0.0, 0.4540272, 0.0, 0.796164...","[italchimica, base, tintometro, smalto, fer, p...","[sistema, tintometrico, base, smalto, antirugg..."
2,maurer cerniera per mobili perno fisso art 500...,confezione 4 pzmisura 1 3 4misura 2 20x20 mm. ...,_@cerniera_bandella_,0.105055,0.512814,0.000000,0.309017,0.000000,0.519669,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.617254,0.000000,"[0.10505541, 0.5128138, 0.0, 0.3090166, 0.0, 0...","[maurer, cerniera, mobili, perno, fisso, art, ...","[confezione, pzmisura, cerniera, mobili, perno..."
3,sanifri 470018043 connettori ad angolo zincati...,sanifri 470018043 connettori ad angolo zincati...,_@staffa_squadretta_,0.052270,0.000000,0.000000,0.006339,0.000000,0.204171,0.434331,...,0.0,0.0,0.0,0.0,0.0,0.019491,0.000000,"[0.052270055, 0.0, 0.0, 0.0063390434, 0.0, 0.2...","[sanifri, connettori, angolo, zincati, senza, ...","[sanifri, connettori, angolo, zincati, senza, ..."
4,weller 2020 saldatore elettrico per ferro 20 w...,weller 2020 saldatore elettrico per ferro 20 w...,_@accessori_saldatore_,0.481601,0.500977,0.133263,0.558505,0.042294,0.000000,0.049805,...,0.0,0.0,0.0,0.0,0.0,0.304132,0.006778,"[0.48160058, 0.5009767, 0.13326323, 0.5585048,...","[weller, saldatore, elettrico, ferro, volt]","[weller, saldatore, elettrico, ferro, volt]"


In [288]:
import nltk 
nltk.download('wordnet') # WordNet® is a large lexical database of English. 
nltk.download('stopwords') 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer # para transformar palabrasa a su raiz
from nltk.stem import WordNetLemmatizer # 

tokenizer = TfidfVectorizer().build_tokenizer() #Return a function that splits a string into a sequence of tokens considering unicode characters
stemmer = SnowballStemmer("italian") 
lemmatizer = WordNetLemmatizer()

##stop-words: el vectorizador ya las remueve pero para mostrar las palabras más frecuente tiene sentido activarlo
remove_stops_here = True

def my_pre_processer(text, debug = False):
    results = []
    if (debug):
        print("Tokenizer result: ", tokenizer(text))
    for token in tokenizer(text):
        clean_token = token.lower().strip('-').strip('_')
        if remove_stops_here and (clean_token in stopwords.words('italian')):
          continue
        #token_pro = stemmer.stem(clean_token) #podemos probar stemming en vez de lematizacion
        token_pro = lemmatizer.lemmatize(clean_token) 
        if len(token_pro) > 2 and not token_pro[0].isdigit(): #elimina palabra largo menor a 2
            results.append(token_pro)
    return results

[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [289]:
data['prep_title'] = data['title'].apply(my_pre_processer)

data['prep_desc'] = data['description'].apply(my_pre_processer)



In [290]:
test_data['prep_title'] = test_data['title'].apply(my_pre_processer)

test_data['prep_desc'] = test_data['description'].apply(my_pre_processer)


In [291]:
used_data = data[['class','prep_title','prep_desc','im_vec']]
used_data['text'] = used_data['prep_title']+ used_data['prep_desc']
#used_data['text'].apply(lambda x: list(set(x)))


In [292]:
used_data.head()


,class,prep_title,prep_desc,im_vec,text
0,_@morsetto_strettoio_,"[record, irwin, irwin, record, morsa, banco, t...","[irwin, record, morsa, banco, tubi]","[0.0, 0.0, 0.0, 0.2543257, 0.0, 0.24739994, 0....","[record, irwin, irwin, record, morsa, banco, t..."
1,_@vernice_,"[italchimica, base, tintometro, smalto, fer, p...","[sistema, tintometrico, base, smalto, antirugg...","[0.4205849, 0.0, 0.0, 0.4540272, 0.0, 0.796164...","[italchimica, base, tintometro, smalto, fer, p..."
2,_@cerniera_bandella_,"[maurer, cerniera, mobili, perno, fisso, art, ...","[confezione, pzmisura, cerniera, mobili, perno...","[0.10505541, 0.5128138, 0.0, 0.3090166, 0.0, 0...","[maurer, cerniera, mobili, perno, fisso, art, ..."
3,_@staffa_squadretta_,"[sanifri, connettori, angolo, zincati, senza, ...","[sanifri, connettori, angolo, zincati, senza, ...","[0.052270055, 0.0, 0.0, 0.0063390434, 0.0, 0.2...","[sanifri, connettori, angolo, zincati, senza, ..."
4,_@accessori_saldatore_,"[weller, saldatore, elettrico, ferro, volt]","[weller, saldatore, elettrico, ferro, volt]","[0.48160058, 0.5009767, 0.13326323, 0.5585048,...","[weller, saldatore, elettrico, ferro, volt, we..."


In [293]:
test_data = test_data[['prep_title','prep_desc']]
test_data['text'] = test_data['prep_title']+ test_data['prep_desc']
test_data['text'].apply(lambda x: list(set(x)))


0                   [plus, banco, durlach, morsa, magnat]
1       [universali, sicutool, largh, installatori, ba...
2       [largh, sicutool, banco, max, peso, apertura, ...
3       [freddo, calore, utilizzo, contenitori, europa...
4           [sta026727, set, decorare, stanley, pennelli]
                              ...                        
1678    [professionali, universali, wiha, lunghezze, v...
1679    [pezzi, metallo, colore, copertura, confezione...
1680    [invisibile, uso, safe, oli, bombola, interno,...
1681                             [millimetri, grani, kwb]
1682                            [girevole, vise, meister]
Name: text, Length: 1683, dtype: object

In [294]:
bigram_converter = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[2,2], lowercase=False) 
x2 = bigram_converter.fit_transform(used_data['text'])
bigrams = bigram_converter.get_feature_names()
trigram_converter = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[3,3], lowercase=False) 
x3 = trigram_converter.fit_transform(used_data['text'])
trigrams = trigram_converter.get_feature_names()


In [295]:
training_data, testing_data = sklearn.model_selection.train_test_split(used_data, train_size = 0.8, random_state=42)

In [296]:
print(training_data.shape)
print(testing_data.shape)

(2019, 5)
(505, 5)


In [297]:
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[3,3], lowercase=False)
X_tr_bow = bow_transform.fit_transform(training_data['text'])

In [298]:
X_te_bow = bow_transform.transform(testing_data['text'])
y_tr = training_data['class']
y_te = testing_data['class']


In [299]:
tfidf_transform = text.TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_transform.transform(X_te_bow)

In [300]:
def simple_logistic_classify(X_tr, y_tr, X_test, y_test,_C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', score)
    return model

In [301]:
model_tfidf = simple_logistic_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te)

Test Score with 0.6831683168316832


In [302]:
def perceptron_classify(X_tr, y_tr, X_test, y_test):
    model = Perceptron(tol = 1e-3,random_state= 0)
    model.fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', score)
    return model

In [303]:
model_perceptron = perceptron_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te)

Test Score with 0.7108910891089109


In [304]:
from sklearn.neighbors import NearestCentroid

In [305]:
model = NearestCentroid()
model.fit(X_tr_tfidf, y_tr)

NearestCentroid()

In [306]:
model.score(X_te_tfidf, y_te)

0.6257425742574257

In [307]:
from sklearn.model_selection import cross_val_score

In [308]:
def cross_val_logistic_classify(X, y,_C=1.0):
    model = LogisticRegression(C=_C).fit(X, y)
    scores = cross_val_score(model, X, y, cv=5)
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
    return model


In [309]:
def cross_val_perceptron_classify(X, y):
    model = Perceptron(tol = 1e-3,random_state= 0)
    model.fit(X, y)
    scores = cross_val_score(model,X,y)
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
    return model

In [310]:
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[3,3], lowercase=False)
X_bow = bow_transform.fit_transform(used_data['text'])
tfidf_transform = text.TfidfTransformer(norm=None)
X = tfidf_transform.fit_transform(X_bow)
y= used_data['class']



In [311]:
X_im = used_data['im_vec']

In [ ]:
cross_val_perceptron_classify(X,y)
